### References:

* https://github.com/pushshift/api
* https://search.pushshift.io/reddit/

In [1]:
import datetime
import pandas as pd
import requests
import spacy

from spacy_langdetect import LanguageDetector
from spacy.language import Language

In [2]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

/usr/local/lib/python3.8/site-packages/spacy/util.py:709: UserWarning: [W095] Model 'en_core_web_sm' (2.3.1) requires spaCy >=2.3.0,<2.4.0 and is incompatible with the current version (3.0.6). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
keyword = 'singapore%20mental%20health'

data = requests.get('https://api.pushshift.io/reddit/submission/search/?q={}&sort=desc&sort_type=score&size=500'.format(keyword))
information = json.loads(data.text)

In [ ]:
result = []

for _information in information['data']:
    subscribers = 0
    subreddit_type = ''
    author_premium = ''
    total_awards_received = 0
    upvote_ratio = 0
    link_flair_text = ''
    
    if 'subreddit_subscribers' in _information.keys():
        subscribers = _information['subreddit_subscribers']
        
    if 'subreddit_type' in _information.keys():
        subreddit_type = _information['subreddit_type']
        
    if 'author_premium' in _information.keys():
        author_premium = _information['author_premium']
        
    if 'total_awards_received' in _information.keys():
        total_awards_received = _information['total_awards_received']
        
    if 'upvote_ratio' in _information.keys():
        upvote_ratio = _information['upvote_ratio']
        
    if 'link_flair_text' in _information.keys():
        link_flair_text = _information['link_flair_text']
        
    if 'selftext' in _information.keys():
        selftext = _information['selftext']
        

    overall_content = _information['title']+"\n\n"+selftext
    doc = nlp(overall_content)
    language_details = doc._.language

    result.append({
      'post_id': _information['id'],
      'subreddit_name':_information['subreddit'],
      'subreddit_id':_information['subreddit_id'], 
      'subreddit_subscribers':subscribers,
      'author':_information['author'], 
      'author_premium':author_premium,
      'subreddit_type':subreddit_type, 
      'num_reports':0, 
      'title':_information['title'], 
      'body':selftext, 
      'overall_content':overall_content,   
      'url':_information['url'],
      'comments': _information['num_comments'],
      'score': _information['score'], 
      'down_votes': 0,
      'up_votes': 0,  
      'upvote_ratio': upvote_ratio,  
      'created_date': datetime.datetime.fromtimestamp(int(_information['created_utc'])).strftime('%c'), 
      'flair_text': link_flair_text,
      'language': language_details['language'],
      'language_score': language_details['score']
    })

posts_df = pd.DataFrame(
    result,
)

display(posts_df.shape)